In [1]:
import os
# I need to visualize all the environment varibale defined
# in the notebook. I need to know the order of the cells
# and the order of the variables defined in each cell.

os.environ.pop("OPENAI_API_BASE", None)


In [2]:
# Import some libraries and especially load environment variables
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# set an environment varibale
os.environ["LANGCHAIN_PROJECT"] = "01_basic"

print(os.getenv("AZURE_ENDPOINT"))
print(os.getenv("OPENAI_API_BASE"))

https://alkopenai2.openai.azure.com/
None


In [3]:
# Create an LLM based on GPT4 deployed on Azure
llm = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt35_2", 
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    model_name="gpt-35-turbo"
)

In [4]:
# I the import my custom function to work with audio and videp
from plugins.CallJarvis import Tasks

tasks = Tasks()

# you can test the function
result = tasks.change_task_title("Task_1", "new title")
print(f"New version of the task: {result}")

Changing title of task Task_1 to new title
New version of the task: 7


In [6]:
# Then create tools list
from langchain.tools import  StructuredTool, Tool

tools = [
    StructuredTool.from_function(
        func=tasks.change_task_title,
        name="ChangeTaskTitle",
        description="Change the title of a task.",
    ),
    StructuredTool.from_function(
        func=tasks.search_task,
        name="SearchTask",
        description="Can search task with a full Text string.",
    ),
    StructuredTool.from_function(
        func=tasks.load_task,
        name="LoadTask",
        description="Given a task_id can load task detail in json format.",
    )
]

In [8]:
# Finally you can create the agent
from langchain.agents import AgentType, initialize_agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [9]:
# Everthing is ready to start the conversation
result = agent.run("""
Search all task containing "verdolazzy" and then change title in "hey this is a new title"
""")

print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "SearchTask",
  "action_input": {
    "search_string": "verdolazzy"
  }
}
```Searching for task verdolazzy

Observation: ['Task_3', 'Task_4', 'Task_6']
Thought:The previous search for tasks containing "verdolazzy" returned the following task IDs: Task_3, Task_4, Task_6. Now, the user wants to change the title of these tasks to "hey this is a new title". To do this, I will use the ChangeTaskTitle tool.
Action:
```
{
  "action": "ChangeTaskTitle",
  "action_input": {
    "task_id": "Task_3",
    "new_title": "hey this is a new title"
  }
}
```
Changing title of task Task_3 to hey this is a new title

Observation: 7
Thought:The title of Task_3 has been successfully changed to "hey this is a new title". Now I need to change the titles of Task_4 and Task_6 as well.
Action:
```
{
  "action": "ChangeTaskTitle",
  "action_input": {
    "task_id": "Task_4",
    "new_title": "hey this is a new title"
  }
}
```
Changing title of ta